# Gensim (день 2)

Импортируем библиотеки:

In [17]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.decomposition import PCA
from matplotlib import pyplot as plt
%matplotlib inline
from sklearn.manifold import TSNE
from gensim.models import Word2Vec
import re

Подготавливаем обучающую выборку:

In [39]:
df = pd.read_csv('Dataset 10.21/products_dns_sorted.csv').dropna()
df_cat = pd.read_csv('Dataset 10.21/categories.csv').dropna()
df['category_id'].replace(df_cat.set_index('category_id').to_dict()['category_name'], inplace=True)
df.rename(columns = {'category_id': 'category'},  inplace = True)
df['description'] = df['description'].str.lower()

desc = df[::100]
d = [x.split() for x in desc]

In [40]:
desc

,product_id,category,description
0,81F1005056A615EF11E309FFE937AB5C,Аксессуары автоэлектроника,зеркало внутрисалонное вега-330 [330 мм; панор...
100,862B005056A615EF11E33C38589C8988,Аксессуары автоэлектроника,ароматизатор ambi-pur антитабак кедровое масло...
200,87E4005056A615EF11E3B15B7096B44D,Аксессуары автоэлектроника,автомобильный диск литой nz f-6 7x16 4/114;3 e...
300,87E4005056A615EF11E3B16C7B50A607,Аксессуары автоэлектроника,автомобильный диск литой dotz shuriken 7x16 4/...
400,87E4005056A615EF11E3B16C964F43DA,Аксессуары автоэлектроника,автомобильный диск литой dotz touge 7x17 4/114...
500,87E4005056A615EF11E3B16CCDEAB47A,Аксессуары автоэлектроника,автомобильный диск литой dotz hammada 8x16 6/1...
600,87E4005056A615EF11E3B16CFF903D31,Аксессуары автоэлектроника,автомобильный диск литой dotz hammada 8x17 6/1...
700,87E4005056A615EF11E3B16D17192F2B,Аксессуары автоэлектроника,автомобильный диск литой dotz nardo 8x19 5/114...
800,87E4005056A615EF11E3B16FBCCF53D1,Аксессуары автоэлектроника,автомобильный диск литой nitro y450 5;5x13 4/9...
900,87E4005056A615EF11E3B170FC23CE63,Аксессуары автоэлектроника,автомобильный диск литой nitro 236 6;5x15 4/98...


In [41]:
words = [[y.lower() for y in x.split() if re.search('\d',y) is None and len(y)>1] for x in desc['description']]
words

[['зеркало', 'внутрисалонное', 'мм;', 'панорамное]'],
 ['ароматизатор', 'ambi-pur', 'антитабак', 'кедровое', 'масло', 'мята'],
 ['автомобильный', 'диск', 'литой', 'nz', 'et', 'dia', 'gmf'],
 ['автомобильный', 'диск', 'литой', 'dotz', 'shuriken', 'et', 'dia', 'dark'],
 ['автомобильный', 'диск', 'литой', 'dotz', 'touge', 'et', 'dia', 'graphite'],
 ['автомобильный', 'диск', 'литой', 'dotz', 'hammada', 'et', 'dia', 'dark'],
 ['автомобильный', 'диск', 'литой', 'dotz', 'hammada', 'et', 'dia', 'dark'],
 ['автомобильный', 'диск', 'литой', 'dotz', 'nardo', 'et', 'dia'],
 ['автомобильный', 'диск', 'литой', 'nitro', 'et', 'dia', 'bfp'],
 ['автомобильный', 'диск', 'литой', 'nitro', 'et', 'dia', 'sil'],
 ['автомобильный', 'диск', 'литой', 'nitro', 'et', 'dia', 'bfp'],
 ['автомобильный', 'диск', 'литой', 'nitro', 'et', 'dia', 'bfp'],
 ['автомобильный', 'диск', 'литой', 'nitro', 'et', 'dia', 'bfp'],
 ['автомобильный', 'диск', 'литой', 'nitro', 'et', 'dia', 'bfpri'],
 ['автомобильный', 'диск', 'литой'

In [44]:
model = Word2Vec(words, size=100, min_count=1, window=5, workers=4)
vocab = model.wv
vocab.similar_by_word('красный')
# vocab.vocab

C:\Users\J\Anaconda3\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('л;', 0.9118413329124451),
 ('до', 0.9093246459960938),
 ('black', 0.9075286388397217),
 ('для', 0.9066605567932129),
 ('сталь;', 0.9066491723060608),
 ('черный', 0.9064294099807739),
 ('гц;', 0.9042569994926453),
 ('вт;', 0.9041779637336731),
 ('bosch', 0.9036985635757446),
 ('usb', 0.9020434617996216)]